In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [13:29<00:00,  1.24it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.594587               0.0      0.808459
std       2.217007               0.0      1.957424
min       0.000007               0.0      0.120016
25%       0.125986               0.0      0.419403
50%       0.657168               0.0      0.792053
75%       2.053709               0.0      0.905938
max      11.428561               0.0     59.568891
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round
model_rel = model.relax()

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model_rel.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rnd_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [15:23<00:00,  1.08it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     10.249950          1.643447      0.922508
std       4.410839          1.653718      0.152595
min       1.965021          0.000000      0.402852
25%       6.754050          0.065237      0.797720
50%       9.723916          1.157247      0.906428
75%      13.284659          2.744793      1.015326
max      21.724260          5.862677      1.445920
Number of infeasible solution: 886


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model_heur.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:11<00:00, 13.89it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    115.844676               0.0      0.071445
std      73.526664               0.0      0.026859
min       7.219464               0.0      0.056119
25%      50.514805               0.0      0.061333
50%     115.428330               0.0      0.062303
75%     150.220972               0.0      0.064135
max     498.093084               0.0      0.188381
Number of infeasible solution: 0


## Heuristic - N3

In [12]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model_heur.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n3_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:13<00:00,  3.95it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      7.665065               0.0      0.252475
std      11.051759               0.0      0.502128
min       0.000007               0.0      0.059156
25%       0.420074               0.0      0.106874
50%       2.392478               0.0      0.188430
75%       9.561415               0.0      0.279285
max      57.349402               0.0     15.028886
Number of infeasible solution: 0


## Learnable Rounding

In [14]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [15]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# define Rosenbrock objective functions and constraints for both problem types
obj_rel, constrs_rel = nmRosenbrock(["x"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmRosenbrock(["x_rnd"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [16]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 95.85124969482422
epoch: 1  train_loss: 42.44343566894531
epoch: 2  train_loss: 25.290496826171875
epoch: 3  train_loss: 20.769079208374023
epoch: 4  train_loss: 16.677532196044922
epoch: 5  train_loss: 13.803874969482422
epoch: 6  train_loss: 12.086336135864258
epoch: 7  train_loss: 11.66338062286377
epoch: 8  train_loss: 10.4960298538208
epoch: 9  train_loss: 10.98294734954834
epoch: 10  train_loss: 8.142115592956543
epoch: 11  train_loss: 8.359756469726562
epoch: 12  train_loss: 7.7702460289001465
epoch: 13  train_loss: 8.44222640991211
epoch: 14  train_loss: 7.556789398193359
epoch: 15  train_loss: 7.439353942871094
epoch: 16  train_loss: 7.1674017906188965
epoch: 17  train_loss: 6.830772399902344
epoch: 18  train_loss: 8.747583389282227
epoch: 19  train_loss: 6.925155162811279
epoch: 20  train_loss: 6.86478328704834
epoch: 21  train_loss: 7.603288173675537
epoch: 22  train_loss: 7.196815013885498
epoch: 23  train_loss: 7.291717052459717
epoch: 24  train_loss:

<All keys matched successfully>

In [17]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_nm_e.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 305.07it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.408696          0.000151      0.002855
std       4.630498          0.002502      0.001063
min       0.018090          0.000000      0.000991
25%       1.199869          0.000000      0.002000
50%       2.534193          0.000000      0.003000
75%       6.262805          0.000000      0.003567
max      35.863158          0.064772      0.009398
Number of infeasible solution: 7
